### ITEM-ITEM 協同過濾：找出與目前瀏覽的商品最相似的商品族群(ITEM-ITEM Similarity Matrix)，推薦給顧客。
- https://towardsdatascience.com/recommender-systems-item-customer-collaborative-filtering-ff0c8f41ae8a

In [1]:
import pandas as pd
df = pd.read_csv('./data/transaction_data_v2.csv')
product_df = pd.read_csv('./data/product_v2.csv')
product_df1 = product_df[['PRODUCT_ID','COMMODITY_DESC']]
df_merge = df.merge(product_df1,on='PRODUCT_ID')
df_merge

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,TRANS_TIME,WEEK_NO,DATE,MEMBER,COMMODITY_DESC
0,2375,26984851472,1,1004906,1,1.39,364,1631,1,2019-07-30,0,POTATOES
1,1929,27021022215,4,1004906,1,1.39,441,1755,1,2019-08-02,1,POTATOES
2,568,27021090189,4,1004906,1,1.39,446,35,1,2019-08-02,0,POTATOES
3,887,27031467254,5,1004906,1,1.39,298,1551,1,2019-08-03,0,POTATOES
4,175,27057508267,7,1004906,1,1.99,446,2303,2,2019-08-05,0,POTATOES
...,...,...,...,...,...,...,...,...,...,...,...,...
2551702,1598,42305362535,711,92130,1,0.99,3228,1520,102,2021-07-09,0,MARGARINES
2551703,1598,42305362535,711,114102,1,8.89,3228,1520,102,2021-07-09,0,BEERS/ALES
2551704,1598,42305362535,711,133449,1,6.99,3228,1520,102,2021-07-09,0,BEERS/ALES
2551705,1598,42305362535,711,6923644,1,4.50,3228,1520,102,2021-07-09,0,BEERS/ALES


In [2]:
df = df_merge.copy()
df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,TRANS_TIME,WEEK_NO,DATE,MEMBER,COMMODITY_DESC
0,2375,26984851472,1,1004906,1,1.39,364,1631,1,2019-07-30,0,POTATOES
1,1929,27021022215,4,1004906,1,1.39,441,1755,1,2019-08-02,1,POTATOES
2,568,27021090189,4,1004906,1,1.39,446,35,1,2019-08-02,0,POTATOES
3,887,27031467254,5,1004906,1,1.39,298,1551,1,2019-08-03,0,POTATOES
4,175,27057508267,7,1004906,1,1.99,446,2303,2,2019-08-05,0,POTATOES


In [3]:
df_new = df[['household_key','BASKET_ID','COMMODITY_DESC','QUANTITY']]

In [4]:
df_new.info()
df_new.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551707 entries, 0 to 2551706
Data columns (total 4 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   household_key   int64 
 1   BASKET_ID       int64 
 2   COMMODITY_DESC  object
 3   QUANTITY        int64 
dtypes: int64(3), object(1)
memory usage: 97.3+ MB


,household_key,BASKET_ID,COMMODITY_DESC,QUANTITY
0,2375,26984851472,POTATOES,1


In [5]:
household_count = df_new.groupby('household_key').BASKET_ID.nunique()
household_count = pd.DataFrame(household_count)
household_count.head()

household_product_count  = df_new.groupby(['COMMODITY_DESC','household_key']).QUANTITY.sum().reset_index().sort_values('COMMODITY_DESC',ascending=False) # Group together
household_product_count.head()

ratings = household_product_count.merge(household_count,on='household_key')
ratings['rating'] = (ratings['QUANTITY']/ratings['BASKET_ID'])

bins = [0,0.005,0.008,0.01,0.02,9]
def rating(value, bins):
    for i in range(len(bins) - 1):
        if value >= bins[i] and value <= bins[i+1]:
            return i+1
    return 1

ratings['rating_label'] = ratings.rating.apply(lambda i: rating(i, bins))

In [6]:
ratings

,COMMODITY_DESC,household_key,QUANTITY,BASKET_ID,rating,rating_label
0,YOGURT,2500,118,100,1.18,5
1,WATER - CARBONATED/FLVRD DRINK,2500,8,100,0.08,5
2,WAREHOUSE SNACKS,2500,31,100,0.31,5
3,VITAMINS,2500,6,100,0.06,5
4,VEGETABLES SALAD,2500,1,100,0.01,3
...,...,...,...,...,...,...
284167,COOKIES/CONES,1897,1,1,1.00,5
284168,CITRUS,1897,1,1,1.00,5
284169,BEERS/ALES,1897,1,1,1.00,5
284170,BAKING NEEDS,1897,1,1,1.00,5


In [7]:
DataBinary = ratings[['household_key','COMMODITY_DESC','rating_label']]
DataBinary

,household_key,COMMODITY_DESC,rating_label
0,2500,YOGURT,5
1,2500,WATER - CARBONATED/FLVRD DRINK,5
2,2500,WAREHOUSE SNACKS,5
3,2500,VITAMINS,5
4,2500,VEGETABLES SALAD,3
...,...,...,...
284167,1897,COOKIES/CONES,5
284168,1897,CITRUS,5
284169,1897,BEERS/ALES,5
284170,1897,BAKING NEEDS,5


In [8]:
DataBinary.describe()

,household_key,rating_label
count,284172.000000,284172.000000
mean,1260.252203,4.529795
std,722.562593,0.981627
min,1.000000,1.000000
25%,638.000000,4.000000
50%,1258.000000,5.000000
75%,1896.000000,5.000000
max,2500.000000,5.000000


In [9]:
purchase_data=DataBinary

In [10]:
purchase_data['COMMODITY_DESC'] = purchase_data['COMMODITY_DESC'].astype(str) 
DfMatrix = pd.pivot_table(purchase_data, values='rating_label', index='household_key', columns='COMMODITY_DESC')
DfMatrix.head()

/home/spark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


COMMODITY_DESC,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
household_key,,,,,,,,,,,,,,,,,,,,,
1,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
2,5.0,5.0,5.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,...,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,5.0,5.0
3,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,...,NaN,NaN,5.0,5.0,NaN,5.0,NaN,NaN,5.0,5.0
4,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN
5,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN


### Normalization reduces the scale of the data to be in a range from 0 to 1:
- Xnormalized = X−Xmin / (Xmax−Xmin)

In [11]:
DfMatrix=DfMatrix.fillna(0) 
#NaN values need to get replaced by 0, meaning they have not been purchased yet.
DfMatrixNorm3 = (DfMatrix-DfMatrix.min())/(DfMatrix.max()-DfMatrix.min())
DfMatrixNorm3.head()

COMMODITY_DESC,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
household_key,,,,,,,,,,,,,,,,,,,,,
1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
DfResetted = DfMatrix.reset_index().rename_axis(None, axis=1) 
DfResetted.head() 
#Now each row represents one customer`s buying behaviour: 1 means the customer has purchased, NaN the customer has not yet purchased it

,household_key,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
0,1,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
1,2,5.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,...,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0
2,3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
3,4,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
4,5,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [13]:
df=DfResetted 

In [14]:
df=df.fillna(0) 
df.head()

,household_key,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
0,1,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
1,2,5.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,...,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0
2,3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
3,4,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
4,5,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [15]:
DfSalesItem = df.drop('household_key', axis=1) 
DfSalesItem.head()

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
0,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
1,5.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,...,0.0,5.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0
2,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0
3,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


### So we can now calculate the Item based recommendation. We will normalize dataframe for comparising reasons:

In [16]:
import numpy as np
DfSalesItemNorm = DfSalesItem/np.sqrt(np.square(DfSalesItem).sum(axis=0))   
DfSalesItemNorm.head()

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
0,0.000000,0.032881,0.030736,0.000000,0.0,0.024776,0.0,0.0,0.000000,0.000000,...,0.0,0.023857,0.021759,0.02636,0.043063,0.027413,0.0,0.000000,0.000000,0.000000
1,0.106819,0.032881,0.030736,0.000000,0.0,0.024776,0.0,0.0,0.043297,0.000000,...,0.0,0.023857,0.021759,0.00000,0.000000,0.027413,0.0,0.000000,0.023773,0.024141
2,0.000000,0.032881,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.043297,0.036149,...,0.0,0.000000,0.021759,0.02636,0.000000,0.027413,0.0,0.000000,0.023773,0.024141
3,0.000000,0.032881,0.030736,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.021759,0.00000,0.000000,0.027413,0.0,0.000000,0.000000,0.000000
4,0.000000,0.032881,0.000000,0.045443,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.021759,0.00000,0.000000,0.000000,0.0,0.053992,0.000000,0.000000


### Calculating with .dot Vectors to compute Cosine Similarities

In [17]:
ItemItemSim = DfSalesItemNorm.transpose().dot(DfSalesItemNorm) 
ItemItemSim.head()

,ADULT INCONTINENCE,AIR CARE,ANALGESICS,ANTACIDS,APPAREL,APPLES,AUDIO/VIDEO PRODUCTS,AUTOMOTIVE PRODUCTS,BABY FOODS,BABY HBC,...,VEAL,VEGETABLES - ALL OTHERS,VEGETABLES - SHELF STABLE,VEGETABLES SALAD,VITAMINS,WAREHOUSE SNACKS,WATCHES/CALCULATORS/LOBBY,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT
ADULT INCONTINENCE,1.000000,0.212141,0.200535,0.219022,0.116099,0.197641,0.115443,0.139864,0.095459,0.147971,...,0.060906,0.219466,0.208533,0.174577,0.214176,0.190098,0.024346,0.136109,0.205290,0.206811
AIR CARE,0.212141,1.000000,0.610728,0.445809,0.303968,0.626225,0.358476,0.395429,0.366612,0.519946,...,0.118808,0.618223,0.656348,0.567847,0.463865,0.587890,0.101171,0.353209,0.645013,0.621166
ANALGESICS,0.200535,0.610728,1.000000,0.527907,0.300461,0.660458,0.375722,0.403588,0.399920,0.565315,...,0.117375,0.648610,0.697718,0.613801,0.523821,0.631470,0.081962,0.341586,0.692783,0.664965
ANTACIDS,0.219022,0.445809,0.527907,1.000000,0.226211,0.496830,0.269062,0.312307,0.257907,0.396559,...,0.133168,0.489552,0.503218,0.435407,0.406027,0.457876,0.039358,0.252324,0.482222,0.472340
APPAREL,0.116099,0.303968,0.300461,0.226211,1.000000,0.303131,0.253504,0.220897,0.169544,0.255496,...,0.050956,0.313376,0.319128,0.280033,0.256756,0.308496,0.057507,0.137819,0.314346,0.319204


In [18]:
def get_similar_products(COMMODITY_DESC):
    similar_score = ItemItemSim.index
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score
# print(get_similar_products('P1000002',5))

In [19]:
ItemNeighbours = pd.DataFrame(index=ItemItemSim.columns,columns=range(1,12))
ItemNeighbours.head()

,1,2,3,4,5,6,7,8,9,10,11
ADULT INCONTINENCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR CARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANALGESICS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANTACIDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APPAREL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
for i in range(0,len(ItemItemSim.columns)): 
    ItemNeighbours.iloc[i,:6] = ItemItemSim.iloc[0:,i].sort_values(ascending=False)[:6].index 
    #we only have 9 items, so we can max recommend 9 items (itself included) 

ItemNeighbours

,1,2,3,4,5,6,7,8,9,10,11
ADULT INCONTINENCE,ADULT INCONTINENCE,TROPICAL FRUIT,DISHWASH DETERGENTS,ORAL HYGIENE PRODUCTS,HOUSEHOLD CLEANG NEEDS,LAXATIVES,NaN,NaN,NaN,NaN,NaN
AIR CARE,AIR CARE,HOUSEHOLD CLEANG NEEDS,LAUNDRY DETERGENTS,FD WRAPS/BAGS/TRSH BG,BATH TISSUES,CONDIMENTS/SAUCES,NaN,NaN,NaN,NaN,NaN
ANALGESICS,ANALGESICS,FD WRAPS/BAGS/TRSH BG,EGGS,CHEESE,BAG SNACKS,CANDY - PACKAGED,NaN,NaN,NaN,NaN,NaN
ANTACIDS,ANTACIDS,ANALGESICS,ORAL HYGIENE PRODUCTS,SOUP,BAG SNACKS,EGGS,NaN,NaN,NaN,NaN,NaN
APPAREL,APPAREL,STATIONERY & SCHOOL SUPPLIES,MAKEUP AND TREATMENT,COOKIES/CONES,ORAL HYGIENE PRODUCTS,CANDY - PACKAGED,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
WAREHOUSE SNACKS,WAREHOUSE SNACKS,BAG SNACKS,CRACKERS/MISC BKD FD,COOKIES/CONES,SOFT DRINKS,CANDY - PACKAGED,NaN,NaN,NaN,NaN,NaN
WATCHES/CALCULATORS/LOBBY,WATCHES/CALCULATORS/LOBBY,DRY NOODLES/PASTA,VEGETABLES - SHELF STABLE,SPICES & EXTRACTS,SNACKS,HOSIERY/SOCKS,NaN,NaN,NaN,NaN,NaN
WATER,WATER,WATER - CARBONATED/FLVRD DRINK,YOGURT,CANNED JUICES,REFRGRATD JUICES/DRNKS,TROPICAL FRUIT,NaN,NaN,NaN,NaN,NaN
WATER - CARBONATED/FLVRD DRINK,WATER - CARBONATED/FLVRD DRINK,SOFT DRINKS,CHEESE,BAG SNACKS,FLUID MILK PRODUCTS,BAKED BREAD/BUNS/ROLLS,NaN,NaN,NaN,NaN,NaN


In [26]:
ItemNeighbours.to_csv("./output_data/a.csv",index=False)

In [21]:
Item_CF = ItemNeighbours.iloc[:,1:6]

In [22]:
Item_CF['recommendations']=Item_CF.values.tolist()

/home/spark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
Item_CF = pd.DataFrame(Item_CF['recommendations'])
Item_CF = Item_CF.reset_index().rename(columns={"index": 'COMMODITY_DESC'})

In [24]:
Item_CF

,COMMODITY_DESC,recommendations
0,ADULT INCONTINENCE,"[TROPICAL FRUIT, DISHWASH DETERGENTS, ORAL HYG..."
1,AIR CARE,"[HOUSEHOLD CLEANG NEEDS, LAUNDRY DETERGENTS, F..."
2,ANALGESICS,"[FD WRAPS/BAGS/TRSH BG, EGGS, CHEESE, BAG SNAC..."
3,ANTACIDS,"[ANALGESICS, ORAL HYGIENE PRODUCTS, SOUP, BAG ..."
4,APPAREL,"[STATIONERY & SCHOOL SUPPLIES, MAKEUP AND TREA..."
...,...,...
299,WAREHOUSE SNACKS,"[BAG SNACKS, CRACKERS/MISC BKD FD, COOKIES/CON..."
300,WATCHES/CALCULATORS/LOBBY,"[DRY NOODLES/PASTA, VEGETABLES - SHELF STABLE,..."
301,WATER,"[WATER - CARBONATED/FLVRD DRINK, YOGURT, CANNE..."
302,WATER - CARBONATED/FLVRD DRINK,"[SOFT DRINKS, CHEESE, BAG SNACKS, FLUID MILK P..."


In [25]:
Item_CF.to_csv("./output_data/Item_CF.csv",index=False)

In [26]:
df = pd.read_csv("./output_data/Item_CF.csv",header=0)

In [27]:
import pymysql
import csv
import sys

host = '3.113.29.214'  # '3.113.29.214'
user = 'eric'  # 'eric'
passwd = '123456'  # '123456'
port = 3306
conninfo = {'host' : host ,'port' : port,'user' : user , 'passwd' : passwd, 'db' : 'recommendation_system','charset' : 'utf8mb4'}


def add_csv(path):
    try:
        conn = pymysql.connect(**conninfo)
        cursor = conn.cursor()
        cursor.execute("""DROP TABLE IF EXISTS item_item_CF""")
        cursor.execute("""CREATE TABLE item_item_CF (commodity_desc longtext, recommendations longtext)""")
        csv_data = csv.reader(open(path))
#         next(csv_data, None)
        for row in csv_data:
            if csv_data.line_num == 1:
                continue
            cursor.execute('INSERT INTO item_item_CF(commodity_desc, recommendations)'\
               'VALUES("%s", "%s")',row)
        # close the connection to the database.
        conn.commit()
        print("Done")
    except:
        print('異常')
        print(sys.exc_info()[0])
        print(sys.exc_info()[1])
    finally:
        cursor.close()
        conn.close()
        print("db close")

In [28]:
csv_data = csv.reader(open("./output_data/Item_CF.csv"))

In [29]:
add_csv('./output_data/Item_CF.csv')

Done
db close
